In [116]:
pip install pulp

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
!pip install -q pyomo

In [119]:
!apt-get install -y -qq glpk-utils

In [120]:
# Importamos Bibliotecas

In [187]:
import pandas as pd
import pyomo.environ as pyo

In [188]:
# Información de los Productos

In [189]:
tipogasolina = {
    'Regular'  : {'precio': 5.73, 'octanaje': 87, 'PVmin': 0.0, 'PVmax': 15.0, 'benceno': 0.8},
    'Especial' : {'precio': 5.94, 'octanaje': 91, 'PVmin': 0.0, 'PVmax': 15.0, 'benceno': 0.8},
    'PremiumX32': {'precio': 6.25, 'octanaje': 93, 'PVmin': 0.0, 'PVmax': 15.0, 'benceno': 0.4},
}

In [190]:
print(pd.DataFrame.from_dict(tipogasolina).T)

            precio  octanaje  PVmin  PVmax  benceno
Regular       5.73      87.0    0.0   15.0      0.8
Especial      5.94      91.0    0.0   15.0      0.8
PremiumX32    6.25      93.0    0.0   15.0      0.4


In [191]:
# Información de las corrientes de refinería

In [192]:
corrientes = {
    'Butano'       : {'NOT': 93.0, 'NOM': 92.0, 'PV': 54.0, 'benceno': 0.00, 'costo': 1.76, 'disponible': 471000},
    'NaftaL'       : {'NOT': 78.0, 'NOM': 76.0, 'PV': 11.2, 'benceno': 0.73, 'costo': 4.25, 'disponible': 54950},
    'Isomerato'    : {'NOT': 83.0, 'NOM': 81.1, 'PV': 13.5, 'benceno': 0.00, 'costo': 4.56, 'disponible': 25000},
    'Reformado'    : {'NOT':100.0, 'NOM': 88.2, 'PV':  3.2, 'benceno': 1.85, 'costo': 5.81, 'disponible': 94200},
    'ReformadoLB'  : {'NOT': 93.7, 'NOM': 84.0, 'PV':  2.8, 'benceno': 0.12, 'costo': 5.70, 'disponible': 7500},
    'NaftaFCC'     : {'NOT': 92.1, 'NOM': 77.1, 'PV':  1.4, 'benceno': 1.06, 'costo': 5.39, 'disponible': 109900},
    'Alquilato'    : {'NOT': 97.3, 'NOM': 95.9, 'PV':  4.6, 'benceno': 0.00, 'costo': 5.70, 'disponible': 62800},
    'aditivo X32'  : {'NOT': 0, 'NOM': 0, 'PV':  3.7, 'benceno': 0, 'costo': 25.65, 'disponible': 500}
}

In [193]:
# Calculemos el octanaje de carretera (NOT + NOM)/2

In [194]:
for c in corrientes.keys():
    corrientes[c]['octanaje'] = (corrientes[c]['NOT'] + corrientes[c]['NOM'])/2

In [195]:
# Imprimamos la información por corrientes:

In [196]:
print(pd.DataFrame.from_dict(corrientes).T)

               NOT   NOM    PV  benceno  costo  disponible  octanaje
Butano        93.0  92.0  54.0     0.00   1.76    471000.0     92.50
NaftaL        78.0  76.0  11.2     0.73   4.25     54950.0     77.00
Isomerato     83.0  81.1  13.5     0.00   4.56     25000.0     82.05
Reformado    100.0  88.2   3.2     1.85   5.81     94200.0     94.10
ReformadoLB   93.7  84.0   2.8     0.12   5.70      7500.0     88.85
NaftaFCC      92.1  77.1   1.4     1.06   5.39    109900.0     84.60
Alquilato     97.3  95.9   4.6     0.00   5.70     62800.0     96.60
aditivo X32    0.0   0.0   3.7     0.00  25.65       500.0      0.00


In [197]:
# Creación del modelo

In [198]:
modelo = pyo.ConcreteModel()

In [199]:
# Variables de Decisión
# Número de Galones tanto para las corrientes como para los tipos de gasolina

In [200]:
C = corrientes.keys()
C

dict_keys(['Butano', 'NaftaL', 'Isomerato', 'Reformado', 'ReformadoLB', 'NaftaFCC', 'Alquilato', 'aditivo X32'])

In [201]:
G = tipogasolina.keys()
G

dict_keys(['Regular', 'Especial', 'PremiumX32'])

In [202]:
#se define que las variables de decision tienen que ser reales positivos
modelo.x = pyo.Var(C,G, domain = pyo.NonNegativeReals)

In [203]:
# Función Objetivo
# Contribución = Ingresos - Costos
# Lo que vamos a hacer a continuación es un poco complicado si no lo entendemos
# Para obtener los ingresos, vamos a multiplicar los galones de cada una de las gasolinas
# por su respectivo precio así:
# GalonesEspecial*PrecioEspecial + GalonesRegular*PrecioRegular
# Recordemos: Las gasolinas no son mas que las mezclas de sus componentes
# Entonces los GalonesEspecial será igual a la sumatoria de todas las corrientes (MP) asignadas
# a la gasolina especial de acuerdo a nuestra optimización.
# Entonces tenemos dos sumatorias:
# La sumatoria de los galones de MP que componen cada una de las gasolinas
# La sumatoria de los ingresos (galones*precio/galón) de cada una de las gasolinas
# Para simplificar la expresión, utilicemos un lazo for en nuestra doble sumatoria


In [204]:
ingresos = sum(sum(modelo.x[c,g]*tipogasolina[g]['precio'] for c in C) for g in G)

In [205]:
costos = sum(sum(modelo.x[c,g]*corrientes[c]['costo'] for c in C) for g in G)

In [206]:
modelo.contribucion = pyo.Objective(expr = ingresos - costos, sense = pyo.maximize)

In [207]:
# Restricciones
# Disponibilidad de la Materia Prima
# Octanaje de acuerdo al tipo gasolina (mayor o igual a lo establecido por tipo de gasolina)
# Presión de Vapor Mínima (mayor o igual a lo establecido por tipo de gasolina)
# Presión de Vapor Máxima (menor o igual a lo establecido por tipo de gasolina)
# Benceno (menor o igual a lo establecido por el tipo de gasolina)

# ¿Cómo escribir las restricciones en sintaxis de Python?
# De nuevo, recordemos que la gasolina es simplemente la adición de ingredientes.
# Por lo tanto, pensemos en función de sumatoria
# Un ejemplo concreto: la disponibilidad de Butano es igual a 471,000 galones
# Por lo tanto:
# GalonesButanoRegular + GalonesButanoEspecial <= Disponibilidad de Butano
# En general: para cada uno de las corrientes (mp), la sumatoria de los galones utilizados
# tanto en regular como en especial debe ser menor a la disponibilidad de cada una de las
# corrientes


In [208]:
modelo.restricciones = pyo.ConstraintList()
for c in C:

  modelo.restricciones.add(sum(modelo.x[c,g] for g in G) <= corrientes[c]['disponible'])

modelo.restricciones.add(modelo.x['aditivo X32','PremiumX32'] >= (0.5/100)*corrientes['aditivo X32']['disponible'])

In [209]:
# Para las siguientes restricciones, haremos una suma ponderada (por los galones) de las diferencias entre la característica
# de las corrientes (butano, nafta, isomerato, etc.) y la característica de una de las gasolinas en particular.
# Por ejemplo para el caso del octanaje de la gasolina regular tendríamos:
# GalonesButanoRegular*(OctanajeButano - OctanajeRegular)
# GalonesNaftaLRegular*(OctanajeNafta - OctanajeRegular)
# GalonesIsomeratoRegular*(OctanajeIsomerato - OctanajeRegular)
# ...así sucesivamente hasta completar las 7 corrientes
# Luego hacemos la suma ponderada, que en el caso del Octanaje debería ser positiva, pues nos interesa que la mezcla resultante
# tenga un Octanaje mayor al requerido por la regular
# Sumatoria() >= 0

In [210]:
for g in G:
    modelo.restricciones.add(sum(modelo.x[c,g]*(corrientes[c]['octanaje'] - tipogasolina[g]['octanaje']) for c in C) >= 0)
    modelo.restricciones.add(sum(modelo.x[c,g]*(corrientes[c]['PV'] - tipogasolina[g]['PVmin']) for c in C) >= 0)
    modelo.restricciones.add(sum(modelo.x[c,g]*(corrientes[c]['PV'] - tipogasolina[g]['PVmax']) for c in C) <= 0)
    modelo.restricciones.add(sum(modelo.x[c,g]*(corrientes[c]['benceno'] - tipogasolina[g]['benceno']) for c in C) <= 0)

In [211]:
# Resolver

In [212]:
solucion = pyo.SolverFactory('glpk')
solucion.solve(modelo)

{'Problem': [{'Name': 'unknown', 'Lower bound': 566623.348197116, 'Upper bound': 566623.348197116, 'Number of objectives': 1, 'Number of constraints': 21, 'Number of variables': 24, 'Number of nonzeros': 121, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.007363080978393555}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [213]:
# Resultados

In [214]:
volumen = sum(modelo.x[c,g]() for c in C for g in G)
print("Volumen Total =", round(volumen, 1), " galones")
print("Contribucion Total = $", round(modelo.contribucion(), 1))
print("Contribución por Galón = $", round(modelo.contribucion()/volumen, 2))

Volumen Total = 446587.3  galones
Contribucion Total = $ 566623.3
Contribución por Galón = $ 1.27


In [215]:
# Resultados por Corriente de Refinería
# En esta sesión,conoceremos cuanto de cada una de las materias primas (corrientes) fue utilizado en cada tipo de gasolina.
# Para esto, crearemos un date frame que contenga 7 filas para las materias primas y cinco columnas: la cantidad utilizada
# en la gasolina regular, especial y total (sumatoria de ambas columnas). Además del inventario inicial y el inventario final.
# Haremos uso de dos lazos for para acceder a filas y columnas y de la funcion .loc de pandas para acceder a cada una de las
# celdas donde están las varibales de modelo.x

In [216]:
resultadosxcorriente = pd.DataFrame()
for c in C:
    for g in G:
        resultadosxcorriente.loc[c,g] = round(modelo.x[c,g](), 1)
    resultadosxcorriente.loc[c, 'Total'] = round(sum(modelo.x[c,g]() for g in G), 1)
    resultadosxcorriente.loc[c, 'InvInicial'] = corrientes[c]['disponible']

resultadosxcorriente['InvFinal'] = resultadosxcorriente['InvInicial'] - resultadosxcorriente['Total']

In [217]:
print(resultadosxcorriente)

             Regular  Especial  PremiumX32     Total  InvInicial  InvFinal
Butano       54874.0   22930.5     14430.3   92234.8    471000.0  378765.2
NaftaL       31277.2   23672.8         0.0   54950.0     54950.0       0.0
Isomerato    25000.0       0.0         0.0   25000.0     25000.0       0.0
Reformado    45949.9   41821.4      6428.7   94200.0     94200.0       0.0
ReformadoLB   7500.0       0.0         0.0    7500.0      7500.0       0.0
NaftaFCC     97070.5       0.0     12829.5  109900.0    109900.0       0.0
Alquilato     2871.4   29888.7     30039.9   62800.0     62800.0       0.0
aditivo X32      0.0       0.0         2.5       2.5       500.0     497.5


In [218]:
# Resultados por Tipo de Gasolina
# En esta sesión, estimaremos el volumen, octanaje, presión de vapor y benceno para cada una de los tipos de gasolinas:
# regular y especial de acuerdo a nuestra optimización. Para ello haremos uso de un lazo for y la función loc de pandas

In [219]:
resultadosxtipogasolina = pd.DataFrame()
for g in G:
    resultadosxtipogasolina.loc[g, 'Volumen'] = round(sum(modelo.x[c,g]() for c in C), 1)
    resultadosxtipogasolina.loc[g, 'Octanaje'] = round(sum(modelo.x[c,g]()*corrientes[c]['octanaje'] for c in C)/resultadosxtipogasolina.loc[g, 'Volumen'], 1)
    resultadosxtipogasolina.loc[g, 'PV'] = round(sum(modelo.x[c,g]()*corrientes[c]['PV'] for c in C)/resultadosxtipogasolina.loc[g, 'Volumen'], 1)
    resultadosxtipogasolina.loc[g, 'Benceno'] = round(sum(modelo.x[c,g]()*corrientes[c]['benceno'] for c in C)/resultadosxtipogasolina.loc[g, 'Volumen'], 1)

In [220]:
print(resultadosxtipogasolina)

             Volumen  Octanaje    PV  Benceno
Regular     264543.0      87.0  15.0      0.8
Especial    118313.4      91.0  15.0      0.8
PremiumX32   63730.9      93.0  15.0      0.4
